In [1]:
from keras.layers import Input, Dense
from keras.layers.core import Reshape
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers import merge
from keras.optimizers import Adadelta, RMSprop
import os
import os.path
import numpy as np
from PIL import Image
from numpy import * 
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import scipy.misc
import cv2

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def model():

    input_img = Input(shape=(128,128,1))
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
      
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
       
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    
    up6 = UpSampling2D((2,2))(conv5)
    up6 = merge([up6, conv4], mode='concat', concat_axis=3)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    
    up7 = UpSampling2D((2,2))(conv6)
    up7 = merge([up7, conv3], mode='concat', concat_axis=3)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    
    up8 = UpSampling2D((2,2))(conv7)
    up8 = merge([up8, conv2], mode='concat', concat_axis=3)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    
    up9 = UpSampling2D((2,2))(conv8)
    up9 = merge([up9, conv1], mode='concat', concat_axis=3)
    conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
       
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(conv9)
    
    ada=Adadelta(lr=5.0, rho=0.95, epsilon=1e-08, decay=0.001)
    rms=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.001)
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(loss='mean_squared_error', optimizer=rms)
    return autoencoder

In [3]:
autoencoder=model()

basic_mat=[]
tobe_mat=[]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:34: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:48: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:55: UserWarning: The `m

In [4]:
path1="Data"

Loading Data, Converting it from RGB to Grayscale

In [5]:
for i in range(1,123):
    path_major=path1+'/'+str(i)
    for j in range(1,100):
        img=array(Image.open(path_major+"/"+str(j)+"_.jpg"))
        #print shape(img)
        img = cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
        img=img.reshape(128,128,1)
        basic_mat.append(img)
        tobe_mat.append(img)

In [6]:
data,Label = shuffle(basic_mat,tobe_mat, random_state=2)

Spliting the Train and Test Data

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, Label, test_size=0.2, random_state=2)
X_train = array(X_train)
y_train = array(y_train)
X_test = array(X_test)
y_test = array(y_test)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(9662, 128, 128, 1)
(9662, 128, 128, 1)
(2416, 128, 128, 1)
(2416, 128, 128, 1)


Normalising the Values between 0 and 1

In [8]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.

y_train = y_train.astype('float32') / 255.
y_test = y_test.astype('float32') / 255.
CheckDir = 'sample/'

In [9]:
for epoch in range(1,300):
    
    train_X,train_Y=shuffle(x_train,y_train)
    print ("Epoch is: %d\n" % epoch)
    batch_size=64
    print ("Number of batches: %d\n" % int(len(train_X)/batch_size))
    num_batches=int(len(train_X)/batch_size)
    for batch in range(num_batches):    
        batch_train_X=train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
        batch_train_Y=train_Y[batch*batch_size:min((batch+1)*batch_size,len(train_Y))]
        loss=autoencoder.train_on_batch(batch_train_X,batch_train_Y)
        print ('epoch_num: %d batch_num: %d loss: %f\n' % (epoch,batch,loss))

    autoencoder.save_weights("fabric_autoen.h5")
    if(epoch%2==0):
        x_test,y_test=shuffle(x_test,y_test)
        decoded_imgs=autoencoder.predict(x_test[:2])
        temp = np.zeros([128, 128*3,3])
        temp[:, :128,:1] = x_test[0,:,:,:1]
        temp[:, 128:128*2,:1] = y_test[0,:,:,:1]
        temp[:, 128*2:,:1] = decoded_imgs[0,:,:,:1]
        temp[:,:,1]=temp[:,:,0]
        temp[:,:,2]=temp[:,:,0]
        temp = temp*255
        scipy.misc.imsave(CheckDir + str(epoch) + ".jpg", temp)

Epoch is: 1

Number of batches: 150

epoch_num: 1 batch_num: 0 loss: 0.082459

epoch_num: 1 batch_num: 1 loss: 0.046771

epoch_num: 1 batch_num: 2 loss: 0.022415

epoch_num: 1 batch_num: 3 loss: 0.012953

epoch_num: 1 batch_num: 4 loss: 0.015663

epoch_num: 1 batch_num: 5 loss: 0.014001

epoch_num: 1 batch_num: 6 loss: 0.011463

epoch_num: 1 batch_num: 7 loss: 0.009117

epoch_num: 1 batch_num: 8 loss: 0.011247

epoch_num: 1 batch_num: 9 loss: 0.007665

epoch_num: 1 batch_num: 10 loss: 0.008855

epoch_num: 1 batch_num: 11 loss: 0.007396

epoch_num: 1 batch_num: 12 loss: 0.005782

epoch_num: 1 batch_num: 13 loss: 0.005678

epoch_num: 1 batch_num: 14 loss: 0.004007

epoch_num: 1 batch_num: 15 loss: 0.004301

epoch_num: 1 batch_num: 16 loss: 0.005058

epoch_num: 1 batch_num: 17 loss: 0.003747

epoch_num: 1 batch_num: 18 loss: 0.003108

epoch_num: 1 batch_num: 19 loss: 0.003156

epoch_num: 1 batch_num: 20 loss: 0.003125

epoch_num: 1 batch_num: 21 loss: 0.003238

epoch_num: 1 batch_num: 22 

epoch_num: 2 batch_num: 39 loss: 0.000350

epoch_num: 2 batch_num: 40 loss: 0.000273

epoch_num: 2 batch_num: 41 loss: 0.000263

epoch_num: 2 batch_num: 42 loss: 0.000303

epoch_num: 2 batch_num: 43 loss: 0.000699

epoch_num: 2 batch_num: 44 loss: 0.000988

epoch_num: 2 batch_num: 45 loss: 0.000458

epoch_num: 2 batch_num: 46 loss: 0.000503

epoch_num: 2 batch_num: 47 loss: 0.000551

epoch_num: 2 batch_num: 48 loss: 0.000334

epoch_num: 2 batch_num: 49 loss: 0.000963

epoch_num: 2 batch_num: 50 loss: 0.002539

epoch_num: 2 batch_num: 51 loss: 0.000876

epoch_num: 2 batch_num: 52 loss: 0.000851

epoch_num: 2 batch_num: 53 loss: 0.000451

epoch_num: 2 batch_num: 54 loss: 0.000371

epoch_num: 2 batch_num: 55 loss: 0.000280

epoch_num: 2 batch_num: 56 loss: 0.000351

epoch_num: 2 batch_num: 57 loss: 0.000319

epoch_num: 2 batch_num: 58 loss: 0.000333

epoch_num: 2 batch_num: 59 loss: 0.000517

epoch_num: 2 batch_num: 60 loss: 0.000780

epoch_num: 2 batch_num: 61 loss: 0.000657

epoch_num: 

epoch_num: 3 batch_num: 78 loss: 0.000398

epoch_num: 3 batch_num: 79 loss: 0.000530

epoch_num: 3 batch_num: 80 loss: 0.000705

epoch_num: 3 batch_num: 81 loss: 0.001016

epoch_num: 3 batch_num: 82 loss: 0.000623

epoch_num: 3 batch_num: 83 loss: 0.000459

epoch_num: 3 batch_num: 84 loss: 0.000467

epoch_num: 3 batch_num: 85 loss: 0.000214

epoch_num: 3 batch_num: 86 loss: 0.000434

epoch_num: 3 batch_num: 87 loss: 0.000260

epoch_num: 3 batch_num: 88 loss: 0.000265

epoch_num: 3 batch_num: 89 loss: 0.000234

epoch_num: 3 batch_num: 90 loss: 0.000602

epoch_num: 3 batch_num: 91 loss: 0.000691

epoch_num: 3 batch_num: 92 loss: 0.000463

epoch_num: 3 batch_num: 93 loss: 0.000371

epoch_num: 3 batch_num: 94 loss: 0.000752

epoch_num: 3 batch_num: 95 loss: 0.000701

epoch_num: 3 batch_num: 96 loss: 0.000581

epoch_num: 3 batch_num: 97 loss: 0.000691

epoch_num: 3 batch_num: 98 loss: 0.000216

epoch_num: 3 batch_num: 99 loss: 0.000244

epoch_num: 3 batch_num: 100 loss: 0.000207

epoch_num:

epoch_num: 4 batch_num: 117 loss: 0.000216

epoch_num: 4 batch_num: 118 loss: 0.000257

epoch_num: 4 batch_num: 119 loss: 0.000225

epoch_num: 4 batch_num: 120 loss: 0.000401

epoch_num: 4 batch_num: 121 loss: 0.000558

epoch_num: 4 batch_num: 122 loss: 0.000250

epoch_num: 4 batch_num: 123 loss: 0.000300

epoch_num: 4 batch_num: 124 loss: 0.000718

epoch_num: 4 batch_num: 125 loss: 0.000294

epoch_num: 4 batch_num: 126 loss: 0.000259

epoch_num: 4 batch_num: 127 loss: 0.000530

epoch_num: 4 batch_num: 128 loss: 0.000280

epoch_num: 4 batch_num: 129 loss: 0.000343

epoch_num: 4 batch_num: 130 loss: 0.000669

epoch_num: 4 batch_num: 131 loss: 0.000401

epoch_num: 4 batch_num: 132 loss: 0.000910

epoch_num: 4 batch_num: 133 loss: 0.000218

epoch_num: 4 batch_num: 134 loss: 0.000201

epoch_num: 4 batch_num: 135 loss: 0.000214

epoch_num: 4 batch_num: 136 loss: 0.000325

epoch_num: 4 batch_num: 137 loss: 0.000189

epoch_num: 4 batch_num: 138 loss: 0.000241

epoch_num: 4 batch_num: 139 loss

epoch_num: 6 batch_num: 5 loss: 0.000720

epoch_num: 6 batch_num: 6 loss: 0.000311

epoch_num: 6 batch_num: 7 loss: 0.000347

epoch_num: 6 batch_num: 8 loss: 0.000278

epoch_num: 6 batch_num: 9 loss: 0.000222

epoch_num: 6 batch_num: 10 loss: 0.000254

epoch_num: 6 batch_num: 11 loss: 0.000134

epoch_num: 6 batch_num: 12 loss: 0.000132

epoch_num: 6 batch_num: 13 loss: 0.000242

epoch_num: 6 batch_num: 14 loss: 0.000414

epoch_num: 6 batch_num: 15 loss: 0.000755

epoch_num: 6 batch_num: 16 loss: 0.000170

epoch_num: 6 batch_num: 17 loss: 0.000249

epoch_num: 6 batch_num: 18 loss: 0.000292

epoch_num: 6 batch_num: 19 loss: 0.000191

epoch_num: 6 batch_num: 20 loss: 0.000149

epoch_num: 6 batch_num: 21 loss: 0.000289

epoch_num: 6 batch_num: 22 loss: 0.000293

epoch_num: 6 batch_num: 23 loss: 0.001479

epoch_num: 6 batch_num: 24 loss: 0.001344

epoch_num: 6 batch_num: 25 loss: 0.000535

epoch_num: 6 batch_num: 26 loss: 0.000428

epoch_num: 6 batch_num: 27 loss: 0.000362

epoch_num: 6 bat

...

epoch_num: 294 batch_num: 88 loss: 0.000022

epoch_num: 294 batch_num: 89 loss: 0.000063

epoch_num: 294 batch_num: 90 loss: 0.000053

epoch_num: 294 batch_num: 91 loss: 0.000024

epoch_num: 294 batch_num: 92 loss: 0.000025

epoch_num: 294 batch_num: 93 loss: 0.000022

epoch_num: 294 batch_num: 94 loss: 0.000038

epoch_num: 294 batch_num: 95 loss: 0.000060

epoch_num: 294 batch_num: 96 loss: 0.000027

epoch_num: 294 batch_num: 97 loss: 0.000021

epoch_num: 294 batch_num: 98 loss: 0.000039

epoch_num: 294 batch_num: 99 loss: 0.000029

epoch_num: 294 batch_num: 100 loss: 0.000030

epoch_num: 294 batch_num: 101 loss: 0.000039

epoch_num: 294 batch_num: 102 loss: 0.000028

epoch_num: 294 batch_num: 103 loss: 0.000019

epoch_num: 294 batch_num: 104 loss: 0.000025

epoch_num: 294 batch_num: 105 loss: 0.000030

epoch_num: 294 batch_num: 106 loss: 0.000042

epoch_num: 294 batch_num: 107 loss: 0.000024

epoch_num: 294 batch_num: 108 loss: 0.000023

epoch_num: 294 batch_num: 109 loss: 0.000022



epoch_num: 295 batch_num: 118 loss: 0.000030

epoch_num: 295 batch_num: 119 loss: 0.000028

epoch_num: 295 batch_num: 120 loss: 0.000022

epoch_num: 295 batch_num: 121 loss: 0.000019

epoch_num: 295 batch_num: 122 loss: 0.000042

epoch_num: 295 batch_num: 123 loss: 0.000045

epoch_num: 295 batch_num: 124 loss: 0.000031

epoch_num: 295 batch_num: 125 loss: 0.000019

epoch_num: 295 batch_num: 126 loss: 0.000016

epoch_num: 295 batch_num: 127 loss: 0.000098

epoch_num: 295 batch_num: 128 loss: 0.000199

epoch_num: 295 batch_num: 129 loss: 0.000022

epoch_num: 295 batch_num: 130 loss: 0.000021

epoch_num: 295 batch_num: 131 loss: 0.000023

epoch_num: 295 batch_num: 132 loss: 0.000028

epoch_num: 295 batch_num: 133 loss: 0.000031

epoch_num: 295 batch_num: 134 loss: 0.000021

epoch_num: 295 batch_num: 135 loss: 0.000020

epoch_num: 295 batch_num: 136 loss: 0.000020

epoch_num: 295 batch_num: 137 loss: 0.000021

epoch_num: 295 batch_num: 138 loss: 0.000031

epoch_num: 295 batch_num: 139 loss

epoch_num: 296 batch_num: 148 loss: 0.000042

epoch_num: 296 batch_num: 149 loss: 0.000023

Epoch is: 297

Number of batches: 150

epoch_num: 297 batch_num: 0 loss: 0.000044

epoch_num: 297 batch_num: 1 loss: 0.000023

epoch_num: 297 batch_num: 2 loss: 0.000033

epoch_num: 297 batch_num: 3 loss: 0.000026

epoch_num: 297 batch_num: 4 loss: 0.000064

epoch_num: 297 batch_num: 5 loss: 0.000030

epoch_num: 297 batch_num: 6 loss: 0.000029

epoch_num: 297 batch_num: 7 loss: 0.000016

epoch_num: 297 batch_num: 8 loss: 0.000016

epoch_num: 297 batch_num: 9 loss: 0.000050

epoch_num: 297 batch_num: 10 loss: 0.000016

epoch_num: 297 batch_num: 11 loss: 0.000029

epoch_num: 297 batch_num: 12 loss: 0.000049

epoch_num: 297 batch_num: 13 loss: 0.000033

epoch_num: 297 batch_num: 14 loss: 0.000019

epoch_num: 297 batch_num: 15 loss: 0.000016

epoch_num: 297 batch_num: 16 loss: 0.000020

epoch_num: 297 batch_num: 17 loss: 0.000035

epoch_num: 297 batch_num: 18 loss: 0.000052

epoch_num: 297 batch_num

epoch_num: 298 batch_num: 28 loss: 0.000028

epoch_num: 298 batch_num: 29 loss: 0.000024

epoch_num: 298 batch_num: 30 loss: 0.000021

epoch_num: 298 batch_num: 31 loss: 0.000025

epoch_num: 298 batch_num: 32 loss: 0.000016

epoch_num: 298 batch_num: 33 loss: 0.000024

epoch_num: 298 batch_num: 34 loss: 0.000023

epoch_num: 298 batch_num: 35 loss: 0.000032

epoch_num: 298 batch_num: 36 loss: 0.000039

epoch_num: 298 batch_num: 37 loss: 0.000065

epoch_num: 298 batch_num: 38 loss: 0.000044

epoch_num: 298 batch_num: 39 loss: 0.000016

epoch_num: 298 batch_num: 40 loss: 0.000021

epoch_num: 298 batch_num: 41 loss: 0.000034

epoch_num: 298 batch_num: 42 loss: 0.000018

epoch_num: 298 batch_num: 43 loss: 0.000016

epoch_num: 298 batch_num: 44 loss: 0.000021

epoch_num: 298 batch_num: 45 loss: 0.000020

epoch_num: 298 batch_num: 46 loss: 0.000022

epoch_num: 298 batch_num: 47 loss: 0.000021

epoch_num: 298 batch_num: 48 loss: 0.000050

epoch_num: 298 batch_num: 49 loss: 0.000018

epoch_num:

epoch_num: 299 batch_num: 59 loss: 0.000019

epoch_num: 299 batch_num: 60 loss: 0.000032

epoch_num: 299 batch_num: 61 loss: 0.000055

epoch_num: 299 batch_num: 62 loss: 0.000021

epoch_num: 299 batch_num: 63 loss: 0.000021

epoch_num: 299 batch_num: 64 loss: 0.000017

epoch_num: 299 batch_num: 65 loss: 0.000015

epoch_num: 299 batch_num: 66 loss: 0.000022

epoch_num: 299 batch_num: 67 loss: 0.000019

epoch_num: 299 batch_num: 68 loss: 0.000063

epoch_num: 299 batch_num: 69 loss: 0.000022

epoch_num: 299 batch_num: 70 loss: 0.000084

epoch_num: 299 batch_num: 71 loss: 0.000017

epoch_num: 299 batch_num: 72 loss: 0.000027

epoch_num: 299 batch_num: 73 loss: 0.000054

epoch_num: 299 batch_num: 74 loss: 0.000032

epoch_num: 299 batch_num: 75 loss: 0.000015

epoch_num: 299 batch_num: 76 loss: 0.000042

epoch_num: 299 batch_num: 77 loss: 0.000045

epoch_num: 299 batch_num: 78 loss: 0.000043

epoch_num: 299 batch_num: 79 loss: 0.000027

epoch_num: 299 batch_num: 80 loss: 0.000016

epoch_num: